In [1]:
using LinearAlgebra, Parameters

# Solving HJB for neoclassical growth models using implicit method

Presented by Chiyoung Ahn (@chiyahn) -- based on Ben Moll's notes in http://www.princeton.edu/~moll/HACTproject/HACT_Additional_Codes.pdf and  http://www.princeton.edu/~moll/HACTproject/HJB_NGM_implicit.m


# Setup

## Production function

In [2]:
A_productivity = 1.0
α = 0.3 
F(k) = A_productivity*k^α;

## Utility function

In [3]:
γ = 2.0
u(c) = c^(1-γ)/(1-γ) 
u_prime(c) = c^(-γ)

u_prime (generic function with 1 method)

## Consumption function by inverse

In [4]:
c(v_prime) = v_prime^(-1/γ); # consumption by derivative of value function at certain k

## Parameters and grids

In [5]:
# ρ: utility discount rate
# δ: capital discount rate
# γ: CRRA parameter
# F: production function that maps k to a real number
# u: utility function that maps c to a real number
# u_prime: derivative of utility function that maps c to a real number
# c: consumption function that maps v_prime (derivative of v at certain k) to a real number
params = (ρ = 0.05, δ = 0.05, γ = γ, F = F, u = u, u_prime = u_prime, c = c)

# ks: grids for states (k) -- assume uniform grids
# maxit: maximum number of iterations
k_ss = (α*A_productivity/(params.ρ+params.δ))^(1/(1-α))
settings = (ks = range(0.001*k_ss, stop = 2*k_ss, length = 10000),
            maxit = 100)

(ks = 0.0048039866566730914:0.0009604129739663476:9.607973313346182, maxit = 100)

# Solve

In [7]:
@unpack ρ, δ, γ, F, u, u_prime, c = params
@unpack ks, maxit = settings

v = (A_productivity .* settings.ks .^ α) .^ (1-params.γ) / (1-params.γ) / params.ρ; # initial guess

P = length(ks)
Δ = ks[2] - ks[1] # assume uniform grids

# begin iterations
for n in 1:maxit
    dv = diff(v) ./ Δ
    dv_f = [dv; NaN] # forward difference
    dv_b = [NaN; dv] # backward difference
    dv_0 = u_prime.(F.(ks) - δ.*ks)
    
    # define the corresponding drifts
    drift_f = F.(ks) - δ.*ks - c.(dv_f)
    drift_b = F.(ks) - δ.*ks - c.(dv_b)

    # steady states at boundary
    drift_f[end] = 0.0
    drift_b[1] = 0.0
    
    # compute consumptions and corresponding u(v)
    I_f = drift_f .> 0.0
    I_b = drift_b .< 0.0
    I_0 = 1 .- I_f-I_b
    
    dv_upwind = dv_f.*I_f + dv_b.*I_b + dv_0.*I_0;
    us = u.(c.(dv_upwind))

    # define the matrix A
    drift_f_upwind = max.(drift_f, 0.0) ./ Δ
    drift_b_upwind = min.(drift_b, 0.0) ./ Δ
    A = LinearAlgebra.Tridiagonal(-drift_b_upwind[2:P], 
            (-drift_f_upwind + drift_b_upwind), 
            drift_f_upwind[1:(P-1)]) 
    
    v_new = (Diagonal(fill((ρ + 1/Δ), P)) - A) \ (us + v ./ Δ)
    @show maximum(abs.(v - v_new))    
    v = v_new
end

maximum(abs.(v - v_new)) = 0.8988941724921489
maximum(abs.(v - v_new)) = 0.8736260133753291
maximum(abs.(v - v_new)) = 0.8493901328830731
maximum(abs.(v - v_new)) = 0.8261346350700194
maximum(abs.(v - v_new)) = 0.8038105994681359
maximum(abs.(v - v_new)) = 0.782371894799482
maximum(abs.(v - v_new)) = 0.761775005058908
maximum(abs.(v - v_new)) = 0.741978867114554
maximum(abs.(v - v_new)) = 0.7229447190345724
maximum(abs.(v - v_new)) = 0.7046359584027044
maximum(abs.(v - v_new)) = 0.6870180099383134
maximum(abs.(v - v_new)) = 0.6700582017830641
maximum(abs.(v - v_new)) = 0.6537256498623236
maximum(abs.(v - v_new)) = 0.6379911497695758
maximum(abs.(v - v_new)) = 0.6228270756619878
maximum(abs.(v - v_new)) = 0.6082072856899572
maximum(abs.(v - v_new)) = 0.5941070335180001
maximum(abs.(v - v_new)) = 0.5805028855237424
maximum(abs.(v - v_new)) = 0.5673726432923871
maximum(abs.(v - v_new)) = 0.5546952710492832
maximum(abs.(v - v_new)) = 0.5424508276988149
maximum(abs.(v - v_new)) = 0.53062040